In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter, defaultdict, namedtuple
%matplotlib inline

In [2]:
np.random.seed(1)

In [3]:
doc = [0,0,2,2,2,4]

In [4]:
counter = Counter(doc)
vocab_size = max(counter) + 1
counts = np.zeros((vocab_size,))
for i, c in counter.items():
    counts[i] += c
counts

array([ 2.,  0.,  3.,  0.,  1.])

In [5]:
np.outer(counts, counts)

array([[ 4.,  0.,  6.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  9.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  3.,  0.,  1.]])

In [6]:
np.outer(counts, counts) - np.diag(counts**2)

array([[ 0.,  0.,  6.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  0.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  3.,  0.,  0.]])

In [7]:
unit_vec = list(np.eye(vocab_size))

In [9]:
H = np.zeros((vocab_size, vocab_size))
for i in doc:
    for j in doc:
        if i != j:
            H[i,j] += 1
    H[i,i] += 1
H

array([[ 2.,  0.,  6.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  3.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 2.,  0.,  3.,  0.,  1.]])

In [12]:
from io import StringIO

In [20]:
x = np.loadtxt(StringIO(u"""\
.048 .042 .047 .024 .029 .026
.035 .040 .045 .016 .023 .029
.031 .019 .031 .040 .045 .042
.027 .023 .031 .032 .039 .045
.047 .043 .035 .026 .021 .019
"""))
x

array([[ 0.048,  0.042,  0.047,  0.024,  0.029,  0.026],
       [ 0.035,  0.04 ,  0.045,  0.016,  0.023,  0.029],
       [ 0.031,  0.019,  0.031,  0.04 ,  0.045,  0.042],
       [ 0.027,  0.023,  0.031,  0.032,  0.039,  0.045],
       [ 0.047,  0.043,  0.035,  0.026,  0.021,  0.019]])

In [23]:
x.sum()

1.0000000000000002

In [27]:
c = np.loadtxt(StringIO(u"""
.33 .14
.29 .12
.02 .33
.05 .29
.32 .11
"""))
s = np.loadtxt(StringIO(u"""
.39 0
0 .61
"""))
h = np.loadtxt(StringIO(u"""
.27 .14
.28 .09
.25 .15
.07 .18
.06 .22
.06 .23
"""))

In [37]:
diff = c.dot(s).dot(h.T) - x
abs_diff = np.abs(diff)
i = np.argmax(abs_diff)
i = np.unravel_index(i, abs_diff.shape)
i, diff[i], np.max(abs_diff)

((4, 2), 0.0062649999999999997, 0.0062649999999999997)

In [41]:
np.sum(c.dot(s).dot(h.T))

0.9998999999999999